**[PT]** Português

---

**[EN]** English

# Bloco de experiências com linked data

---

# Notebook to experiment with linked data


## Exemplo de pesquisas

---

## Query examples

### Estudantes da Universidade de Coimbra nascidos antes do século XX

---

### Students of the University of Coimbra born before the 20th century

2022-05-18 : 454 results stored in [inferences/wikidata](../inferences/wikidata/query.json)

In [33]:
# Id of Universidade de Coimbra
uc_qid = 'Q368643'

# Propriedade "educated at"
p_educated = 'P69'

# Birth
p_birth_date = 'P569'
p_birth_place = 'P19'

# Death
p_death_date = 'P570'
p_death_place = 'P20'

# country
p_country = 'P27'

# Biblioteca Nacional ID
p_bnp_id = 'P1005'


### Obter todos os estudantes da UC, com data e local de nascimento e morte, e id na Biblioteca Nacional

---

### Fetch all the students of UC with date and place of birth and deadth and Portuguese National Library ID

# qwikidata



In [34]:
!pip install qwikidata


In [35]:
all_query = """

SELECT DISTINCT ?student ?bnp_idLabel ?studentLabel ?countryLabel ?birthDateLabel ?birthPlace ?birthPlaceLabel ?deathDateLabel ?deathPlace ?deathPlaceLabel 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "pt". }
  {
    SELECT DISTINCT ?student ?bnp_id ?country ?birthDate ?birthPlace ?deathDate ?deathPlace WHERE {
      ?student p:P69 ?statement0.
      ?statement0 (ps:P69/(wdt:P279*)) wd:Q368643.
      ?student p:P569 ?statement_1.
      OPTIONAL { ?student p:P570/psv:P570/wikibase:timeValue ?deathDate.}
      OPTIONAL { ?student p:P27/ps:P27 ?country.}
      OPTIONAL { ?student p:P19/ps:P19 ?birthPlace.}
      OPTIONAL { ?student p:P20/ps:P20 ?deathPlace.}
      OPTIONAL { ?student p:P1005/ps:P1005 ?bnp_id.}
      ?statement_1 psv:P569 ?statementValue_1.
      ?statementValue_1 wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?statementValue_1 wikibase:timeValue ?birthDate.
      FILTER(?birthDate< "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    LIMIT 1000
  }
}

"""

Query Wikidata

In [36]:
from qwikidata.sparql import return_sparql_query_results

results = return_sparql_query_results(all_query)
records = results['results']['bindings']


Show sample records

In [37]:
records[:2]

[{'student': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q64697672'},
  'studentLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'José Luís Alves Feijó'},
  'birthDateLabel': {'type': 'literal', 'value': '1816-01-08T00:00:00Z'},
  'deathDateLabel': {'type': 'literal', 'value': '1874-11-07T00:00:00Z'},
  'deathPlace': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q768261'},
  'birthPlace': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q1999803'},
  'countryLabel': {'xml:lang': 'pt', 'type': 'literal', 'value': 'Portugal'},
  'birthPlaceLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'Freixo de Espada à Cinta'},
  'deathPlaceLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'Bragança'}},
 {'student': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q24013847'},
  'studentLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'José Lages Perestrelo de Vasconcelos'},
  'birthDateLabel': {'type'

Create dataframe from results

In [43]:
import pandas as pd


tuples = [(  
            student.get('student',{'value':None})['value'],
            student.get('studentLabel',{'value':None})['value'],
            student.get('bnp_idLabel',{'value':None})['value'],
            student.get('birthPlaceLabel',{'value':None})['value'],
            student.get('birthPlace',{'value':None})['value'],
            student.get('birthDateLabel',{'value':'0000-00-00'})['value'][0:10], # keep just the date, the time is normally missing
            )
            for student in records]

students = pd.DataFrame(tuples, columns=['wikidata','name','bnp_id','naturalidade','placeID','data_nasc'])


In [44]:

students.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wikidata      455 non-null    object
 1   name          455 non-null    object
 2   bnp_id        277 non-null    object
 3   naturalidade  421 non-null    object
 4   placeID       421 non-null    object
 5   data_nasc     455 non-null    object
dtypes: object(6)
memory usage: 21.5+ KB


In [50]:
import warnings

from timelinknb.pandas import attribute_to_df
from timelinknb import current_time,current_machine, get_db
from ucalumni.config import default_db
from IPython.display import display

db_spec = default_db
db = get_db(db_spec)
print(current_machine,current_time,f'db={db_spec}')

students.sort_values('naturalidade', inplace=True)
students['fauc_id'] = None

not_found = 0
match = 0
ambiguous = 0

for i,row in students.iterrows():
    uri = row['wikidata']
    name = row['name']
    name_like = name.split(" ")[0]+'%'+name.split(" ")[-1]
    birth_date = row['data_nasc']
    print()
    print(f"Looking for {name} {birth_date} {uri}")
    if row['naturalidade'] is not None:
        place = row['naturalidade']
        print(f"  {place}")
        candidates = attribute_to_df('naturalidade',the_value='%'+place+'%',name_like=name_like,dates_in=(birth_date,'1920-00-00' ))
        if candidates.iloc[0].count() == 0:
            print("Not found")
            not_found = not_found + 1
        else:
            display(candidates.head(5))
            if len(candidates.index) == 1:
                print("Matched")
                fauc_id = candidates.iloc[0].name
                match = match + 1
                students.loc[i]['fauc_id'] = fauc_id
            else:  # check if one name matches exactly
                hit = candidates[candidates['name'] == name]
                if len(hit) == 0:
                    print("Ambiguous")
                    ambiguous = ambiguous + 1
                elif len(hit) > 0:
                    print("Matched")
                    fauc_id = hit.iloc[0].name  # note this 'name' is the index entry
                    match = match + 1
                    students.loc[i]['fauc_id'] = fauc_id
                    if len(hit) > 1 :
                        warnings.warn("Possible duplicates: "+", ".join(hit.index.values))

print(f"Matched {match} ambiguous {ambiguous} not found {not_found}")



mini-m1.local 2022-05-25 18:06:12.641970 db=('sqlite', 'fauc.db')

Looking for António Augusto da Silva Martins 1892-04-04 http://www.wikidata.org/entity/Q18715803
  Abrantes


,name,sex,naturalidade,naturalidade.date,naturalidade.obs
id,,,,,
184909,António Augusto da Silva Martins,m,Abrantes,1910-10-15,None


Matched

Looking for Samora Barros 1887-04-03 http://www.wikidata.org/entity/Q10367297
  Albufeira
Not found

Looking for António de Oliveira Monteiro 1842-01-02 http://www.wikidata.org/entity/Q21290091
  Alcafozes


,name,sex,naturalidade,naturalidade.date,naturalidade.obs
id,,,,,
251928,António de Oliveira Monteiro,m,Alcafozes,1860-07-09,None


Matched

Looking for José Eduardo Raposo de Magalhães 1844-06-12 http://www.wikidata.org/entity/Q10309302
  Alcobaça


...

Matched

Looking for Mário de Figueiredo 1890-04-19 http://www.wikidata.org/entity/Q10334567
  Viseu
Not found

Looking for Q1771793 1588-10-16 http://www.wikidata.org/entity/Q1771793
  Waterford
Not found

Looking for José Marques de Castilho 1869-01-03 http://www.wikidata.org/entity/Q10310060
  Águeda
Not found

Looking for Afonso Pinto Veloso 1878-10-09 http://www.wikidata.org/entity/Q16142675
  Águeda


,name,sex,naturalidade,naturalidade.date,naturalidade.obs
id,,,,,
214665,Afonso de Melo Pinto Veloso,m,Águeda,1893-10-20,None


Matched

Looking for Jacinto Carlos da Silveira 1732-09-18 http://www.wikidata.org/entity/Q64747752
  Évora


,name,sex,naturalidade,naturalidade.date,naturalidade.obs
id,,,,,
230556,Jacinto Carlos da Silveira,m,Évora,1750-10-01,None


Matched

Looking for José Maria de Abreu Freire e Almeida 1827-03-28 http://www.wikidata.org/entity/Q16337692

Looking for Álvaro Augusto Pegado de Sousa Pinto Barroso 1862-04-09 http://www.wikidata.org/entity/Q16489277

Looking for Abílio Beça 1856-08-20 http://www.wikidata.org/entity/Q9561614

Looking for António Roberto de Barros Leitão e Carvalhosa 1763-06-07 http://www.wikidata.org/entity/Q23771067

Looking for António Dinis da Graça Vieira 1824-07-26 http://www.wikidata.org/entity/Q18707730

Looking for Joaquim Desterro de Almeida 1880-02-22 http://www.wikidata.org/entity/Q10307138

Looking for António Caetano Celorico Gil 1880-08-07 http://www.wikidata.org/entity/Q24203205

Looking for António Pires da Silva de Azevedo Loureiro 1833-04-01 http://www.wikidata.org/entity/Q16492263

Looking for Joaquim Pedro Martins 1875-12-23 http://www.wikidata.org/entity/Q16930167

Looking for Bernardo Rodrigues Nogueira 1695-04-06 http://www.wikidata.org/entity/Q16464464

Looking for Francisco 

In [41]:
students.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455 entries, 34 to 338
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          455 non-null    object
 1   bnp_id        277 non-null    object
 2   naturalidade  421 non-null    object
 3   placeID       421 non-null    object
 4   data_nasc     455 non-null    object
 5   fauc_id       114 non-null    object
dtypes: object(6)
memory usage: 41.0+ KB


In [32]:
students.to_csv('../inferences/wikidata/students_wikidata_matched.csv',sep=',')

In [1]:

query = """

    SELECT DISTINCT ?student 
     WHERE {
      ?student p:P69 ?educatedAt.
      ?educatedAt (ps:P69/(wdt:P279*)) wd:Q368643.
      ?student p:P569 ?birthDateStmt.
      ?birthDateStmt psv:P569 ?birthDateValue.
      ?birthDateValue wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?birthDateValue wikibase:timeValue ?P569_1.
      FILTER(?P569_1 < "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    LIMIT 5

"""

In [2]:
!pip install lxml

You should consider upgrading via the '/Users/jrc/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
from datetime import datetime

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import return_sparql_query_results

from lxml import etree

# Prepare to fetch records from Portuguese National Library



xsl_file = '../extras/bnp/visbd-fauc.xsl'
xsl = etree.parse(xsl_file) 

# Things we would like to known about the students
claims_of_interest = ['P69','P1559','P1447','P27','P569','P570','P106','P19','P20',
                      'P22','P25','p97', 
                      'P973',                      # described at URL
                      'P1005','P102','P39','P166','P213','P214']
                      
claims_ignored = set()
prop_labels = {}
cache = {}
results = return_sparql_query_results(query)

# https://qwikidata.readthedocs.io/en/stable/entity.html
for result in results['results']['bindings']:
    uri = result['student']['value']
    qid = uri.split("/")[-1]  # last item in the path
    result_dict = get_entity_dict_from_api(qid)
    std = WikidataItem(result_dict)

    print(std.entity_id,std.get_label())
    for aka in std.get_aliases(lang='pt'):
        print(f"{' '*2} aka:",aka)

    claim_groups = std.get_claim_groups()


    claim_id_list = sorted([c for c in claim_groups.keys() if c in claims_of_interest])
    if 'P1005' in claim_id_list:     # we move the property related to published works to the end
        claim_id_list.remove('P1005')
        claim_id_list = claim_id_list + ['P1005']


    for claim_id in claim_id_list:
        if claim_id in prop_labels.keys():
            propLabel = prop_labels[claim_id]
        else:
            prop_dict = get_entity_dict_from_api(claim_id)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[claim_id] = propLabel

        claim_group = claim_groups[claim_id]
        print(f"{' '*2} {claim_id} {propLabel}({len(claim_group)})")
        for claim in claim_group:
            snak = claim.mainsnak
            if snak.snak_datatype == 'wikibase-item':
                qid = snak.datavalue.value["id"]
                entity = WikidataItem(get_entity_dict_from_api(qid))
                print(f"{' '*6}{qid} {entity.get_label(lang='pt')}")
            elif snak.value_datatype == 'string':
                print(f"{' '*6} {snak.datavalue.value}") 
            elif snak.value_datatype == 'time':
                svalue = snak.datavalue.value['time']
                try:
                    date_value = datetime.strptime(svalue,"+%Y-%m-%dT%H:%M:%SZ")
                    print(f"{' '*6} {date_value}")  
                except ValueError as e:
                    print("Problem converting date, probably date incomplete: "+svalue,e)
            else:
                snak_info = (claim.rank,snak.snaktype,snak.snak_datatype,snak.value_datatype)  
                print(f"{' '*6}{snak_info}")
            
            if claim_id == 'P1005': 
                bnp_id = snak.datavalue.value
                url = f"http://urn.bn.pt/bibliografia/unimarc/xml?id={bnp_id}"
                records = etree.parse(url)
                transform = etree.XSLT(xsl)
                print(str(transform(records)))

            for pid, quals in claim.qualifiers.items():
                prop = WikidataProperty(get_entity_dict_from_api(pid))
                for qual in quals:
                    if qual.snak.snaktype != "value":
                        continue
                    else:
                        print(f"{' '*9}{prop.get_label()} {qual.snak.datavalue}")

    print(f"{' '*3}Other properties:",set(claim_groups.keys())-set(claims_of_interest) )
    claims_ignored = claims_ignored.union(set(claim_groups.keys())-set(claims_of_interest))
    print()

print()
if len(claims_ignored) > 0:
    print("Properties present in the students and ignored in this list")
    for ignored in sorted(claims_ignored):
        if ignored in prop_labels.keys():
            propLabel = prop_labels[prop]
        else:
            prop_dict = get_entity_dict_from_api(ignored)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[prop] = propLabel  
        print(f"   {ignored:<6s}: {propLabel}")



Q61755108 Manuel Augusto Martins
   P106 occupation(1)
      Q82955 político
   P19 place of birth(1)
      Q2078801 Santa Maria Maior
   P20 place of death(1)
      Q25444 Funchal
   P27 country of citizenship(1)
      Q45 Portugal
   P569 date of birth(1)
       1867-04-10 00:00:00
   P570 date of death(1)
       1936-04-04 00:00:00
   P69 educated at(1)
      Q368643 Universidade de Coimbra
         academic major WikibaseEntityId(id=Q7748)
   Other properties: {'P735', 'P21', 'P1412', 'P18', 'P2671', 'P103', 'P31', 'P734'}

Q16917757 Francisco Correia de Mendonça
   P106 occupation(1)
      Q16533 juiz
   P19 place of birth(1)
      Q732548 Lagos
   P27 country of citizenship(1)
      Q45 Portugal
   P569 date of birth(1)
       1832-01-03 00:00:00
   P570 date of death(1)
       1899-08-04 00:00:00
   P69 educated at(1)
      Q368643 Universidade de Coimbra
   Other properties: {'P735', 'P21', 'P1412', 'P2671', 'P103', 'P31'}

Q17272838 Álvaro Rodrigues Azevedo
   aka: José Rodrig

## Lugares

---

## Places


In [ ]:
place = "Soure"

In [ ]:
pquery = """

SELECT DISTINCT ?location ?locationName ?instanceOf ?instanceOfLabel ?insideLabel ?inside2Label ?coordinates ?geonamesID WHERE {
                        ?location ?label '$place'@pt .  
                        ?location wdt:P17 wd:Q45 .
                        ?location rdfs:label ?locationName .
                        ?location wdt:P31 ?instanceOf .
                        ?location wdt:P625 ?coordinates .
                        ?location wdt:P131 ?inside .

                        OPTIONAL {

                          ?inside wdt:P131 ?inside2 .
                          ?location wdt:P1566 ?geonamesID .
                        }
                        FILTER(lang(?locationName) = "pt") 
                        { SERVICE wikibase:label { bd:serviceParam wikibase:language "pt". }}
 
                        }
"""

#### Encontrar um lugar na wikipedia

---

### Find a place in Wikipedia

In [ ]:
!pip install mkwikidata


In [ ]:
import mkwikidata
import pandas as pd

place = 'Soure'

query_result = mkwikidata.run_query(pquery, params={'place':place})
results = query_result['results']['bindings']
table = [{'name':loc['locationName']['value'],
          'type':loc['instanceOfLabel']['value'],
          'uri':loc['location']['value'],
          'inside':loc['insideLabel']['value'],
          'inside2':loc.get('inside2Label',{'value':None})['value'],
          'geonamesID':loc.get('geonamesID',{'value':None})['value'],
          
          } for loc in results]

pd.DataFrame(table).sort_values('uri')

,name,type,uri,inside,inside2,geonamesID
2,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Região de Coimbra,8011895
4,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Região de Coimbra,8011895
5,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Coimbra,8011895
8,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Coimbra,8011895
0,Soure,município de Portugal,http://www.wikidata.org/entity/Q543993,Coimbra,Portugal,2733851
1,Soure,vila de Portugal,http://www.wikidata.org/entity/Q543993,Coimbra,Portugal,2733851
3,Soure,vila de Portugal,http://www.wikidata.org/entity/Q543993,Região de Coimbra,None,None
7,Soure,município de Portugal,http://www.wikidata.org/entity/Q543993,Região de Coimbra,None,None
6,Soure,vila de Portugal,http://www.wikidata.org/entity/Q97597938,Soure,None,None


In [ ]:
from timelinknb import current_time,current_machine, get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)
print(current_machine,current_time,f'db={db_spec}')

imac-jrc.local 2022-05-18 17:43:33.378466 db=('sqlite', 'fauc.db')


### Localizar os lugares mais infrequentes no FAUC

---

### Locate the less frequent places in the FAUC

In [ ]:
from timelinknb.pandas import attribute_values


# Get list of different values and the number of occurences
naturalidades = attribute_values('naturalidade')
small_places = naturalidades[naturalidades['count']<10].sort_values('count', ascending=False).head(100)

Test with a small number

In [ ]:
from IPython.display import display
import time

for place in small_places.index.values[:10]:
    print("Looking for:", place)
    try:
        query_result = mkwikidata.run_query(pquery, params={'place':place})
    except Exception as e:
        print(e)
    results = query_result['results']['bindings']
    if len(results) > 0:
        table = [{'name':loc['locationName']['value'],
                'type':loc['instanceOfLabel']['value'],
                'uri':loc['location']['value'],
                'inside':loc['insideLabel']['value'],
                'inside2':loc.get('inside2Label',{'value':None})['value'],
                'geonamesID':loc.get('geonamesID',{'value':None})['value'],
                
                } for loc in results]
        candidates = pd.DataFrame(table)
        display(candidates)
        geonames = candidates['geonamesID'].unique()
        for geo in geonames:
            print(f"   https://www.geonames.org/{geo}")
    else:
        print("   not found")
        
    time.sleep(1)  # respect wikidata

Looking for: Alcofra


,name,type,uri,inside,inside2,geonamesID
0,Alcofra,freguesia de Portugal,http://www.wikidata.org/entity/Q1021445,Vouzela,Viseu,8014009
1,Alcofra,freguesia de Portugal,http://www.wikidata.org/entity/Q1021445,Vouzela,Comunidade Intermunicipal Viseu Dão Lafões,8014009


   https://www.geonames.org/8014009
Looking for: Lumiar


,name,type,uri,inside,inside2,geonamesID
0,Estação Lumiar,estação subterrânea,http://www.wikidata.org/entity/Q374170,Lisboa,None,None
1,Estação Lumiar,estação de metropolitano,http://www.wikidata.org/entity/Q374170,Lisboa,None,None
2,Lumiar,freguesia de Portugal,http://www.wikidata.org/entity/Q924723,Lisboa,Lisboa,8012475


   https://www.geonames.org/None
   https://www.geonames.org/8012475
Looking for: Seixo de Ansiães


,name,type,uri,inside,inside2,geonamesID
0,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Trás-os-Montes e Alto Douro,8014442
1,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Bragança,8014442
2,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Douro,8014442


   https://www.geonames.org/8014442
Looking for: Santa Cristina de Figueiró


,name,type,uri,inside,inside2,geonamesID
0,Santa Cristina de Figueiró,localidade,http://www.wikidata.org/entity/Q1023987,Amarante,Porto,8012656
1,Santa Cristina de Figueiró,freguesia de Portugal,http://www.wikidata.org/entity/Q1023987,Amarante,Douro Litoral,8012656
2,Santa Cristina de Figueiró,localidade,http://www.wikidata.org/entity/Q1023987,Amarante,Douro Litoral,8012656
3,Santa Cristina de Figueiró,freguesia de Portugal,http://www.wikidata.org/entity/Q1023987,Amarante,Porto,8012656


   https://www.geonames.org/8012656
Looking for: Sandelgas
   not found
Looking for: Salvaterra
   not found
Looking for: Rio Bom
   not found
Looking for: Ribeira de Homem
   not found
Looking for: Quintãs
   not found
Looking for: Proença
   not found


## Biblioteca Nacional de Portugal

---

## Portuguese National Library



Get all the bibiographic records of a student with P1005 id.



In [94]:
from lxml import etree

bnp_id = "92687"
url = f"http://urn.bn.pt/bibliografia/unimarc/xml?id={bnp_id}"

xsl_file = '../extras/bnp/visbd-fauc.xsl'
xsl = etree.parse(xsl_file) 
records = etree.parse(url)
transform = etree.XSLT(xsl)

print(str(transform(records)))





Socialismo libertario e anarchismo : história e doutrina / Silva Mendes. - [S.l. : s.n.], 1896. - 369 p. ; 22 cm
http://id.bnportugal.gov.pt/bib/catbnp/418597

Sobre filosofia / Manuel da Silva Mendes. - Macau : Leal Senado, [1915]. - 169 p. : il. ; 21 cm
http://id.bnportugal.gov.pt/bib/catbnp/1579131

Socialismo libertário ou anarchismo : história e doutrina / Manuel da Silva Mendes. - ed. fac-simil. - Lisboa : Letra Livre, D.L. 2006. - 369 p. : fac-simil. ; 24 cm. - Ed. fac-simil., Lisboa, 1896. - ISBN 989-95114-0-4. - ISBN 978-989-95114-0-8
http://id.bnportugal.gov.pt/bib/catbnp/1563060

Manuel da Silva Mendes : a instrução pública em Macau ; org. e introd. António Aresta. - Macau : Direcção dos Serviços de Educação e Juventude, 1996. - 96, [5] p., [7] p. il. : il. ; 24 cm. - (Educação memórias ; 3). - ISBN 972-8091-32-X
http://id.bnportugal.gov.pt/bib/catbnp/987433

Arte chinesa : (colectânea de artigos) / Manuel da Silva Mendes. - [Macau] : Edição do Leal Senado de Macau, 1983. -

In [16]:
recs = records.getroot()
recs.tag

'{http://www.bn.pt/standards/metadata/marcxml/1.0/}collection'

In [92]:
marcxml_ns = "http://www.bn.pt/standards/metadata/marcxml/1.0/"
nsmap = {None: marcxml_ns}
for rec in recs:
    print("Record:")
    for cf in rec.findall("controlfield",namespaces=nsmap):
        print(cf.get('tag'),cf.text)
    for df in rec.findall("datafield",namespaces=nsmap):
        print(f"{df.get('tag'):3s} {df.get('ind1'):1s}{df.get('ind2'):1s}")
        for sf in list(df):
            print(f"   ${sf.get('code')} {sf.text}",)
print()


Record:
001 418597
003 http://id.bnportugal.gov.pt/bib/catbnp/418597
005 19981231000000.0
095   
   $a PTBN00447303
100   
   $a 19930417d1896    m  y0pory0103    ba
101 0 
   $a por
102   
   $a PT
200 0 
   $a Socialismo libertario e anarchismo
   $e história e doutrina
   $f Silva Mendes
210   
   $a [S.l.
   $c s.n.],
   $d 1896
215   
   $a 369 p.
   $d 22 cm
500 10
   $a Socialismo libertário ou anarquismo
700  1
   $a Mendes,
   $b Silva,
   $f 1867-1931
   $3 92687
801  0
   $a PT
   $b BN
   $g RPC
966   
   $l BN
   $m FGMON
   $s S.C. 3676 V.
966   
   $l BN
   $m FGMON
   $s S.C. 11055 P.
997   
   $a BIBEOD
Record:
001 1579131
003 http://id.bnportugal.gov.pt/bib/catbnp/1579131
100   
   $a 20070321d1915    m  y0pory0103    ba
101 0 
   $a por
102   
   $a MO
105   
   $a a   z   001yy
106   
   $a r
200 1 
   $a Sobre filosofia
   $f Manuel da Silva Mendes
210   
   $a Macau
   $c Leal Senado,
   $d [1915]
215   
   $a 169 p.
   $c il.
   $d 21 cm
675   
   $a 22Taoísmo
  

In [10]:
recs = records['collection']

cfields = recs['record'][0]['controlfield']
dfields = recs['record'][0]['datafield']
cfields


TypeError: 'lxml.etree._ElementTree' object is not subscriptable